In [1]:
import os
from os.path import exists
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax, pad
import math
import copy
import time
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
import altair as alt
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.datasets as datasets
import spacy
import GPUtil
import warnings
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP

# Set to False to skip notebook execution (e.g. for debugging)
warnings.filterwarnings("ignore")
RUN_EXAMPLES = True

In [2]:
# make transformer
def make_model(src_vocab, tgt_vocab, head=6, N=6, d_model=512, f_size=1024, dropout=0.1):
    c = copy.deepcopy
    
    pe = PositionEncoder(d_model, dropout)
    attn = MultiHeadAttention(d_model, head, dropout)
    ff = PositionWiseFeedForward(d_model, f_size, dropout)
    
    model = EncoderDecoder(
        nn.Sequential(Embedding(d_model, src_vocab), c(pe)),
        nn.Sequential(Embedding(d_model, tgt_vocab), c(pe)),
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        Generator(d_model, tgt_vocab)
    )
    
    for param in model.parameters():
        if param.dim() > 1:
            nn.init.xavier_uniform_(param)
    
    return model  

In [3]:
class EncoderDecoder(nn.Module):
    def __init__(self, src_embed, tgt_embed, encoder, decoder, generator):
        super(EncoderDecoder, self).__init__()
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.encoder = encoder
        self.decoder = decoder
        self.generator = generator
    def forward(self, src, tgt, src_mask, tgt_mask):
        return self.decode(tgt, tgt_mask, src_mask, self.encode(src, src_mask))
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    def decode(self, tgt, tgt_mask, src_mask, memory):
        return self.decoder(self.tgt_embed(tgt), tgt_mask, src_mask, memory) 

In [4]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, f_size, dropout=0.1):
        super(PositionWiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, f_size)
        self.w_2 = nn.Linear(f_size, d_model)
        self.dropout = nn.Dropout(p=dropout)
    def forward(self, x):
        return self.w_2(self.dropout(self.w_1(x).relu()))

In [5]:
class Embedding(nn.Module):
    def __init__(self, feat_size, vocab):
        super(Embedding, self).__init__()
        self.lut = nn.Embedding(vocab, feat_size)
        self.feat_size = feat_size
    
    def forward(self, x):
        return self.lut(x) * math.sqrt(self.feat_size)

In [24]:
class PositionEncoder(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionEncoder, self).__init__()
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(-1)
        
        div_term = torch.exp(torch.arange(0, d_model, 2) * -math.log(10000)/d_model)        
        
        pe[:,0::2] = torch.sin(position*div_term) 
        pe[:,1::2] = torch.cos(position*div_term)
        
        pe = pe.unsqueeze(1)
        
        self.register_buffer('pe', pe)
        
        self.dropout = nn.Dropout(p=dropout)
    
    def forward(self, x):
        self.pe[:, :x.size(1)].requires_grad = False
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

In [7]:
def Clone(mod, N):
    return nn.ModuleList([copy.deepcopy(mod) for _ in range(N)])

In [8]:
class LayerNorm(nn.Module):
    def __init__(self, feat_size, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.w_m = nn.parameter.Parameter(torch.ones(feat_size))
        self.w_b = nn.parameter.Parameter(torch.zeros(feat_size))
        self.eps = eps
        
    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        
        return self.w_m*(x-mean)/(std+self.eps) + self.w_b

In [9]:
class SubConnectionLayer(nn.Module):
    def __init__(self, feat_size, dropout=0.1):
        super(SubConnectionLayer, self).__init__()
        self.norm = LayerNorm(feat_size)
        self.dropout = nn.Dropout(p=dropout)
    def forward(self, x, layer):
        layer_out = layer(self.norm(x))
        drop = self.dropout(layer_out)
        x = x + drop
        # x = x + self.dropout(layer(self.norm(x)))
        return x

In [10]:
def Attention(q, k, v, mask=None, dropout=None):
    d_k = q.size(-1)
    
    score = torch.matmul(q, k.transpose(-1,-2)) / math.sqrt(d_k)
    
    if mask is not None:
        score = score.masked_fill(mask==0, -1e9)
    
    attn = score.softmax(-1)
    
    if dropout is not None:
        attn = dropout(attn)
    
    return torch.matmul(attn, v), attn
    
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, head, dropout=0.1):
        super(MultiHeadAttention, self).__init__()
        assert d_model % head == 0
        self.d_k = d_model // head
        self.head = head
        self.dropout = nn.Dropout(p=dropout)
        self.linears = Clone(nn.Linear(d_model, d_model), 4)
        self.attn = None
        
    def forward(self, query, key, value, mask=None):
        nbatch = query.size(0)
        
        if mask is not None:
            # ?? the size of mask
            mask = mask.unsqueeze(1)
        
        # ?? why do we need transpose
        q, k, v = [
            lin(x).view(nbatch, -1, self.head, self.d_k).transpose(1,2) for lin, x in zip(self.linears, (query, key, value))
        ]
        
        # print(f" x before attn {x}")
        x, self.attn = Attention(q, k, v, mask, self.dropout)
        
        x = x.transpose(1,2).contiguous().view(nbatch, -1, self.head*self.d_k)
        
        del q
        del k
        del v
        
        return self.linears[-1](x)
            

In [11]:
class Encoder(nn.Module):
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = Clone(layer, N)
        self.norm = LayerNorm(layer.size)
    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [12]:
class EncoderLayer(nn.Module):
    def __init__(self, feat_size, attn, ff, dropout=0.1):
        super(EncoderLayer, self).__init__()
        self.layers = Clone(SubConnectionLayer(feat_size, dropout), 2)
        self.attn = attn
        self.ff = ff
        self.size = feat_size
    def forward(self, x, mask):
        x = self.layers[0](x, lambda x: self.attn(x,x,x,mask))
        # print(f"x is {x} with {type(x)}")
        return self.layers[1](x, self.ff)

In [13]:
class Decoder(nn.Module):
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = Clone(layer, N)
        self.norm = LayerNorm(layer.size)
    def forward(self, x, tgt_mask, src_mask, memory):
        for layer in self.layers:
            x = layer(x, tgt_mask, src_mask, memory)
        return self.norm(x)

In [23]:
class DecoderLayer(nn.Module):
    def __init__(self, feat_size, self_attn, mem_attn, ff, dropout=0.1):
        super(DecoderLayer, self).__init__()
        self.layers = Clone(SubConnectionLayer(feat_size, dropout), 3)
        self.self_attn = self_attn
        self.mem_attn = mem_attn
        self.ff = ff
        self.size = feat_size
    def forward(self, x, tgt_mask, src_mask, memory):
        m = memory
        x = self.layers[0](x, lambda x: self.self_attn(x,x,x,tgt_mask))
        x = self.layers[1](x, lambda x: self.mem_attn(x,m,m,src_mask))
        return self.layers[2](x, self.ff)

In [15]:
class Generator(nn.Module):
    def __init__(self, d_model, tgt_vocab):
        super(Generator, self).__init__()
        self.map = nn.Linear(d_model, tgt_vocab)
    
    def forward(self, x):
        return log_softmax(self.map(x), dim=-1)

In [20]:
def subsequent_mask(feat_size):
    attn_shape = (1, feat_size, feat_size)
    submask = torch.triu(torch.ones(attn_shape), diagonal=1).type(torch.uint8)
    
    return submask == 0

In [16]:
def show_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        return fn(*args)

In [ ]:
def inference_test():
    test_model = make_model(11, 11, 2)
    test_model.eval()
    src = torch.LongTensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]])
    src_mask = torch.ones(1, 1, 10)

    memory = test_model.encode(src, src_mask)
    ys = torch.zeros(1, 1).type_as(src)
    
    for i in range(9):
        out = test_model.decode(
            ys, subsequent_mask(ys.size(1)).type_as(src.data), src_mask, memory
        )
        # out = test_model.decode(
        #     memory, src_mask, ys, subsequent_mask(ys.size(1)).type_as(src.data)
        # )
        prob = test_model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.data[0]
        ys = torch.cat(
            [ys, torch.empty(1, 1).type_as(src.data).fill_(next_word)], dim=1
        )

    print("Example Untrained Model Prediction:", ys)


def run_tests():
    for _ in range(10):
        inference_test()


show_example(run_tests)